### OpenAI에서 지원하는 Assistants API의 Code Interpreter를 활용하여 PPT 파일 생성
- Assistants : 대화 및 파일 생성, 코드 명령 수행, 데이터 검색 및 추출 등 측정 Task에 특화되어 학습된 자연어 기반 생성형 인공지능 API
- OpenAI의 일반적인 Chat AI처럼 대화형 인공지능 서비스를 제공하지만, Assistants는 특정 작업에 특화하여 지원하거나 자동화에 초점을 두어 챗봇, 가상 비서, 자동 응답 시스템 개발 등에 포커스를 맞춰 활용 가능

In [1]:
from openai import OpenAI
from getpass import getpass
import time

In [2]:
MY_API_KEY = getpass("OpenAI API Key >> ")

In [3]:
client = OpenAI(api_key = MY_API_KEY)

### 어시스턴트 세부 사항 정의

In [4]:
# 역할 지정
assistant_instruction = "PowerPoint 파일을 만들어줘. 너는 자전거 라이딩 전문가이자 PowerPoint 작성 전문가야. 배경, 색상 및 스타일은 읽기 쉽게 만들고, 글꼴은 페이지 별로 제목은 40point 내외, 나머지는 20point 내외로 설정해줘. 슬라이드는 3개 이내로 만들고, 페이지의 텍스트는 한글로 작성 해야돼."

# 사용자의 요청 작성
prompt_user = "개인 기록을 목표로 자전거 라이딩을 연습하는 사람들에게 강의하기 위한 프레젠테이션 자료를 만들어줘. 초, 중, 고급 수준 별에 대해 적절한 라이딩 횟수, 경로, 기본적인 스트레칭 동작에 대한 설명을 포함하는 프레젠테이션을 만들어줘. 깔끔하고 구체적으로 작성해줘. 바로 작성해줘."

### 어시스턴트, 스레드, 메시지 객체 생성
- thread : 프로세스 내에서 실행되는 가장 작은 실행 단위로, 본 실습에서는 어시스턴트와 사용자 간의 채팅 세션을 하나의 스레드로 볼 수 있음
- message : 스레드 내의 통신 단위

#### 1. 어시스턴트 객체 생성

In [5]:
assistant = client.beta.assistants.create(
    name = "My Assistant",
    instructions = assistant_instruction,
    # tools : 설정 가능 타입은 code_interpreter, function, file_search
    tools = [{"type" : "code_interpreter"}],
    model = "gpt-3.5-turbo"
)
assistant

Assistant(id='asst_IP10WfBHSXzitE7Y3QLD5uiU', created_at=1720535642, description=None, instructions='PowerPoint 파일을 만들어줘. 너는 자전거 라이딩 전문가이자 PowerPoint 작성 전문가야. 배경, 색상 및 스타일은 읽기 쉽게 만들고, 글꼴은 페이지 별로 제목은 40point 내외, 나머지는 20point 내외로 설정해줘. 슬라이드는 3개 이내로 만들고, 페이지의 텍스트는 한글로 작성 해야돼.', metadata={}, model='gpt-3.5-turbo', name='My Assistant', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0)

In [6]:
type(assistant)

openai.types.beta.assistant.Assistant

#### 2. 스레드 객체 생성
- 대화를 시작하면 스레드가 생성되며, 해당 대화 중에 교환되는 모든 메시지는 스레드에서 진행

In [7]:
thread = client.beta.threads.create()
thread

Thread(id='thread_Zg1fkIphyhd329qw7ASsJdo0', created_at=1720535656, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

#### 3. 메시지 객체 생성
- 메시지는 사용자 질의, 명령과 같은 텍스트 뿐 아니라 사용자와 어시스턴트 간 교환되는 모든 콘텐츠일 수 있음
- 메시지는 스레드 내의 통신 단위이므로, 위에서 생성한 스레드 객체 id 값을 입력

In [8]:
msg = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = prompt_user
)
msg

Message(id='msg_G2TBtzKxMN6H061Fz9544gFF', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='개인 기록을 목표로 자전거 라이딩을 연습하는 사람들에게 강의하기 위한 프레젠테이션 자료를 만들어줘. 초, 중, 고급 수준 별에 대해 적절한 라이딩 횟수, 경로, 기본적인 스트레칭 동작에 대한 설명을 포함하는 프레젠테이션을 만들어줘. 깔끔하고 구체적으로 작성해줘. 바로 작성해줘.'), type='text')], created_at=1720535839, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_Zg1fkIphyhd329qw7ASsJdo0')

In [9]:
type(thread), type(msg)

(openai.types.beta.thread.Thread, openai.types.beta.threads.message.Message)

#### 4. 스레드 실행 (채팅 창에 입력)
- 채팅을 시작하여 위에 설정된 정보로 역할 설정 및 원하는 질의를 실제로 입력하여 요청하는 단계

In [10]:
# 스레드를 실행하면, LLM 모델로 사용자의 요청이 넘어감
# 또한 모델이 이에 대해 응답을 마치면 스레드 종료
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id
)
run

Run(id='run_q9IT82O2Nta5ti53p7cArl6X', assistant_id='asst_IP10WfBHSXzitE7Y3QLD5uiU', cancelled_at=None, completed_at=None, created_at=1720536313, expires_at=1720536913, failed_at=None, incomplete_details=None, instructions='PowerPoint 파일을 만들어줘. 너는 자전거 라이딩 전문가이자 PowerPoint 작성 전문가야. 배경, 색상 및 스타일은 읽기 쉽게 만들고, 글꼴은 페이지 별로 제목은 40point 내외, 나머지는 20point 내외로 설정해줘. 슬라이드는 3개 이내로 만들고, 페이지의 텍스트는 한글로 작성 해야돼.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_Zg1fkIphyhd329qw7ASsJdo0', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

- 스레드를 실행시킨다고 해서, 설정해 둔 모든 요청과 그에 대한 응답이 즉각적으로 이루어 지지는 않음
- 생성 모델이 사용자의 요청을 인지하고, 결과물을 생성하기 위해서는 어느 정도의 시간이 필요 (복잡한 작업이나 거대한 결과물을 생성할 때는 더욱 시간이 오래걸릴 수 있음)

In [11]:
# 요청에 따른 응답 진행 상황 확인용 코드
interval_time = 3

while True :
    # retrieve() : 특정 스레드의 실행 상태나 실행 후 결과를 생성, 조회하는 함수
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id # 스레드 실행에 대한 id
    )
    # 실행 완료
    # status : completed, failed, in_progress (완료, 실패, 진행 중)
    if run.status == "completed" :
        print("실행 완료!")
        break
    # 실행 중
    else :
        print(run.status)
        # 일정한 텀을 두고 상태 확인 (즉시, 반복적으로 서버에 상태 확인을 요청하면 서버에 부하가 걸릴 수 있음)
        time.sleep(interval_time)

실행 완료!


In [12]:
msgs = client.beta.threads.messages.list(thread_id = thread.id)
msgs

SyncCursorPage[Message](data=[Message(id='msg_K01dntQvetcQkjpBMiacOBid', assistant_id='asst_IP10WfBHSXzitE7Y3QLD5uiU', attachments=[Attachment(file_id='file-fgMYx8MwVK3jbR6FrqG1KN4J', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=130, file_path=FilePath(file_id='file-fgMYx8MwVK3jbR6FrqG1KN4J'), start_index=86, text='sandbox:/mnt/data/Biking_Training_Guide.pptx', type='file_path')], value='프레젠테이션 파일이 성공적으로 생성되었습니다. 아래 링크를 통해 다운로드 받으실 수 있습니다:\n\n[Biking Training Guide 다운로드 링크](sandbox:/mnt/data/Biking_Training_Guide.pptx)\n\n확인해보시고 추가적인 수정이 필요하다면 언제든지 알려주세요.'), type='text')], created_at=1720536328, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_q9IT82O2Nta5ti53p7cArl6X', status=None, thread_id='thread_Zg1fkIphyhd329qw7ASsJdo0'), Message(id='msg_EoNv3tvCSKKDWJLYyVaemS9l', assistant_id='asst_IP10WfBHSXzitE7Y3QLD5uiU', a

In [13]:
msgs.data[0].content[0].text.value

'프레젠테이션 파일이 성공적으로 생성되었습니다. 아래 링크를 통해 다운로드 받으실 수 있습니다:\n\n[Biking Training Guide 다운로드 링크](sandbox:/mnt/data/Biking_Training_Guide.pptx)\n\n확인해보시고 추가적인 수정이 필요하다면 언제든지 알려주세요.'

In [14]:
msgs.data[0].content[0].text.annotations

[FilePathAnnotation(end_index=130, file_path=FilePath(file_id='file-fgMYx8MwVK3jbR6FrqG1KN4J'), start_index=86, text='sandbox:/mnt/data/Biking_Training_Guide.pptx', type='file_path')]

In [15]:
msgs.data[0].content[0].text.annotations[0]

FilePathAnnotation(end_index=130, file_path=FilePath(file_id='file-fgMYx8MwVK3jbR6FrqG1KN4J'), start_index=86, text='sandbox:/mnt/data/Biking_Training_Guide.pptx', type='file_path')

#### annotations가 비어있는 경우, 원인
- 생성 모델이 결과물을 전부 생성하기 전에 코드를 실행했을 경우 -> 충분한 시간이 지난 후 코드 실행
- 필요한 데이터가 메시지에 포함되지 않아 모델이 작업을 수행할 수 없는 경우 (파일 미 첨부, 애매한 지시 등)
- API 호출에 문제가 있거나, 잘못된 thread_id를 사용하여 올바른 데이터를 가져오지 못한 경우

In [16]:
# 메시지를 통해 받은 파일 정보 중 파일의 id 값을 변수에 저장
file_id_path = msgs.data[0].content[0].text.annotations[0].file_path.file_id
# retrieve_content() : file_id 값을 통해 생성된 파일의 내용을 검색해 반환
file_contents = client.files.with_raw_response.retrieve_content(file_id_path)
file_contents

<APIResponse [200 OK] type=<class 'str'>>

#### 결과물을 실제 파일로 내보내기

In [17]:
# 경로 설정
output_file = "../data/ppt_001.pptx"

'''
wb (write binary) : 쓰기 모드 (파일을 생성하거나 기존 파일에 덮어쓸 때 사용)
rb (read binary) : 읽기 모드 (파일을 읽어들일 때 사용)
'''
with open(output_file, "wb") as f :
    # 메시지에서 가져온 내용 중 콘텐츠 내용을 파일에 쓰기 
    f.write(file_contents.content)

#### 자원 해제 : 동작 중인 어시스턴트 중단 및 메모리 삭제
- 요청과 응답이라는 하나의 스레드는 항상 사용자의 요청을 대기하고 있기 때문에 꼭 해제해야 함

In [18]:
client.files.delete(file_id_path)
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_IP10WfBHSXzitE7Y3QLD5uiU', deleted=True, object='assistant.deleted')